In [1]:
import torch;import torch.nn as nn;import torch.nn.functional as F
torch.set_default_tensor_type(torch.cuda.FloatTensor)
from layer.modules.ssd import SSD
from layer.function.iou import match
a = torch.rand(2,3,300,300)
b = SSD()
c = b(a)
d = torch.tensor([[0.0,0.0,1.0,1.0,2.0],[0.5,0.5,1.0,1.0,5.0]]).reshape(2,-1) #fake truthbox
d1 = torch.tensor((0.0,0.0,1.0,1.0,2.0)).reshape(1,-1)

In [2]:
matchbox = torch.cuda.FloatTensor(*c[1].shape)
loc_conf = torch.cuda.FloatTensor(*c[0].shape)

for i,j in enumerate((d,d1)):
    e = match(c[2],j)
    matchbox[i].zero_().scatter_(1,e[1].unsqueeze(1),1.0)
    loc_conf[i] = e[0]

In [3]:
print(matchbox.shape,loc_conf.shape)

torch.Size([2, 8732, 21]) torch.Size([2, 8732, 4])


In [4]:
num_neg = torch.clamp(matchbox.long().sum(1,keepdim=True) * 3,max=matchbox.size(1)-1)

In [5]:
print(num_neg)

tensor([[[   0,    0,   39,    0,    0,  384,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0, 8731]],

        [[   0,    0,   39,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0, 8731]]])


In [6]:
loss_c = c[1].clone()
pos = matchbox == 1
loss_c[pos] = 0

In [12]:
_, loss_idx = loss_c.sort(1,descending=True)
_, idx_rank = loss_idx.sort(1)
neg = idx_rank < num_neg.expand_as(idx_rank)

In [16]:
neg[:,:,0]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]],
       dtype=torch.uint8)

In [15]:
neg[:,-1]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]],
       dtype=torch.uint8)

In [17]:
neg.shape

torch.Size([2, 8732, 21])